In [1]:
import numpy as np
from scipy.optimize import least_squares
#from pandas import Series, DataFrame
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Qt5Agg')
%matplotlib qt5
#
# if pade.py is not in the current directory, set this path:
#
#import sys
#sys.path.append('../Python_libs')
from rac_aux import *

In [3]:
Angs2Bohr=1.8897259886
au2eV=27.211386027
au2cm=219474.63068
#
#  files in the current directory do not need the path name
#
#df = pd.read_csv("/home/thomas/Python/StabPlots/Stab_data/1D_a0.2_b0_c0.14/crossing_1.dat", delim_whitespace=True)
df = pd.read_csv("sb_rac.csv")
#df = pd.read_csv("crossing_1.dat", delim_whitespace=True)
plt.cla()
plt.plot(df.l.values, df.E1.values, 'o-')
plt.plot(df.l.values, df.E2.values, 'o-')
plt.plot(df.l.values, df.E3.values, 'o-')
plt.show()
df[:5]

,l,E1,E2,E3,E4
0,0.0,0.469219,1.365173,1.938043,3.271038
1,0.2,0.457846,1.314401,1.844238,3.242423
2,0.4,0.446073,1.251127,1.760926,3.215267
3,0.6,0.433754,1.173608,1.690292,3.189317
4,0.8,0.420648,1.082215,1.632448,3.164365


In [4]:
i_neg = np.argmin(abs(df.E1.values))
if df.E1[i_neg] > 0:
    i_neg += 1
ls = df.l.values[i_neg:]
print('N=',len(ls))
Es = df.E1.values[i_neg:]
plt.cla()
plt.plot(df.l.values, df.E1.values, 'b-')
plt.plot(df.l.values, df.E2.values, 'b-')
plt.plot(df.l.values, df.E3.values, 'b-')
plt.plot(ls, Es, 'o', color="orange")
plt.show()

N= 89


In [5]:
k2s = -Es
ks  = np.sqrt(k2s)

In [23]:
# start params
E0 = linear_extra(ls,Es)
G0 = 0.2*E0
p0s=[ls[0]] + guess(E0, G0) + guess(5*E0,10*G0) + [10]
print(p0s)
print(chi2_gen(p0s, ks, k2s, ls, pade_42_lsq))

[2.4, 0.2704669890140844, 1.4666963639216455, 0.570916223186097, 3.29172828739109, 10]
91.97737298598086


In [ ]:
p31_opt = [2.40228618, 0.27139798, 1.28131923]

In [ ]:
#
#  the derivative of [4,2] works
#
#pade_31(ks, k2s, p0s)
#df1s = pade_31_num_jac(ks[87:], k2s[87:], p0s, step=1e-2, tiny=1e-4)
#print("num grad:", df1s)

df2s = pade_31j_lsq(p0s, ks[-8:], k2s[-8:], ls)
print("ana grad:", df2s)

In [ ]:
# start params
E0 = linear_extra(ls,Es)
G0 = 0.2*E0
p0s=[ls[0]] + guess(E0, G0) + [10]
print(chi2_gen(p0s, ks, k2s, ls, pade_31))

In [27]:
#
#  test results of other minimizers wrt least squares
#
#p0s=res.x

print("Start parameters:",p0s)
print('Least squares, 2-point jac')
res = least_squares(pade_42_lsq, p0s, method='trf', jac='2-point', args=(ks, k2s, ls))
print("njev:",res.njev)
print("cost:",res.cost)
print("grad:",res.grad)
print("message:",res.message)
print("success:",res.success)
print("x:", res.x)
print('chi2 = %.3e' % (res.cost*2/len(ks)))
print("Er=%f,  Gamma=%f" % res_ene(res.x[1], res.x[2]))

Start parameters: [2.4, 0.2704669890140844, 1.4666963639216455, 0.570916223186097, 3.29172828739109, 10]
Least squares, 2-point jac
njev: 131
cost: 9.866093195108429e-07
grad: [-3.22149497e-08 -2.27616111e-09  7.42028049e-08 -8.66504336e-09
 -5.62722762e-08  4.93020439e-08]
message: `ftol` termination condition is satisfied.
success: True
x: [2.39193215 0.29644137 1.31873181 1.37363888 0.29655248 0.50789533]
chi2 = 2.217e-08
Er=1.731331,  Gamma=0.463547


In [ ]:
print('Least squares, trust-region-reflective (default) with analytic jac')
res = least_squares(pade_31_lsq, p0s, method='trf', jac=pade_31j_lsq, args=(ks, k2s, ls))
print("njev:",res.njev)
print("cost:",res.cost)
print("grad:",res.grad)
print("message:",res.message)
print("success:",res.success)
print("x:", res.x)
print('chi2 = %.3e' % (res.cost*2/len(ks)))
print("Er=%f,  Gamma=%f" % res_ene(res.x[1], res.x[2]))

In [ ]:
print('Least squares, Levenberg-Marquardt with analytic jac')
res = least_squares(pade_31_lsq, p0s, method='lm', jac=pade_31j_lsq, args=(ks, k2s, ls))
print("njev:",res.njev)
print("cost:",res.cost)
print("grad:",res.grad)
print("message:",res.message)
print("success:",res.success)
print("x:", res.x)
print('chi2 = %.3e' % (res.cost*2/len(ks)))
print("Er=%f,  Gamma=%f" % res_ene(res.x[1], res.x[2]))

In [ ]:
print('Least squares, TRF with bounds')

bnds=([0,0,0,0],[np.inf,np.inf,np.inf,np.inf])

res = least_squares(pade_31_lsq, p0s, jac=pade_31j_lsq, bounds=bnds, args=(ks, k2s, ls))
print("njev:",res.njev)
print("cost:",res.cost)
print("grad:",res.grad)
print("message:",res.message)
print("success:",res.success)
print("x:", res.x)
print('chi2 = %.3e' % (res.cost*2/len(ks)))
print("Er=%f,  Gamma=%f" % res_ene(res.x[1], res.x[2]))

In [ ]:
l0, a, b, d = res.x